In [1]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 11.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importer les bibliothèques nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# Pour le ML
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, learning_curve, validation_curve, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Charger le jeu de données
path = "/home/onyxia/work/statapp_sujet26/"

file_name1 = "dataset_complet_avec_dummies_part_1.csv"
file_name2 = "dataset_complet_avec_dummies_part_2.csv"
file_name3 = "dataset_complet_avec_dummies_part_3.csv"
file_name4 = "dataset_complet_avec_dummies_part_4.csv"
df1 = pd.read_csv(path+file_name1, sep=',',low_memory=False)
df2 = pd.read_csv(path+file_name2, sep=',',low_memory=False)
df3 = pd.read_csv(path+file_name2, sep=',',low_memory=False)
df4 = pd.read_csv(path+file_name2, sep=',',low_memory=False)
df = pd.concat([df1,df2,df3,df4])


df['grav'] = df['grav'].replace({1:0,2:0,3:1,4:1})

y = df['grav']
X = df.drop(columns=['grav'])
features = X.columns


# Diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normaliser les données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Définir une grille de paramètres à rechercher
param_grid = {
    'max_depth': [5, 15, 25],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [20, 100, 200],
}

model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

###GRID SEARCH POUR F1###
grid_search_f1 = GridSearchCV(estimator=model, param_grid=param_grid, cv=k_fold, scoring='f1')
grid_search_f1.fit(X_train, y_train)
# Afficher les meilleurs paramètres et le score f1
print("Best Parameters (f1):", grid_search_f1.best_params_)
model_f1 = grid_search_f1.best_estimator_

y_pred = model_f1.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("Score F1 sur l'ensemble de test:", f1)


###GRID SEARCH POUR Accuracy###
grid_search_acc = GridSearchCV(estimator=model, param_grid=param_grid, cv=k_fold, scoring='accuracy')
grid_search_acc.fit(X_train, y_train)
# Afficher les meilleurs paramètres et le score accuracy
print("Best Parameters (Accuracy):", grid_search_acc.best_params_)
model_acc = grid_search_acc.best_estimator_

y_pred = model_acc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy sur l'ensemble de test:", accuracy)

###GRID SEARCH POUR Recall###
grid_search_recall = GridSearchCV(estimator=model, param_grid=param_grid, cv=k_fold, scoring='recall')
grid_search_recall.fit(X_train, y_train)
# Afficher les meilleurs paramètres et le score recall
print("Best Parameters (Recall):", grid_search_recall.best_params_)
model_recall = grid_search_recall.best_estimator_

y_pred = model_recall.predict(X_test)
recall = recall_score(y_test, y_pred)
print("Recall sur l'ensemble de test:", recall)

###GRID SEARCH POUR Precision###
grid_search_precision = GridSearchCV(estimator=model, param_grid=param_grid, cv=k_fold, scoring='precision')
grid_search_precision.fit(X_train, y_train)
# Afficher les meilleurs paramètres et le score precision
print("Best Parameters (Precision):", grid_search_precision.best_params_)
model_precision = grid_search_precision.best_estimator_

y_pred = model_precision.predict(X_test)
precision = precision_score(y_test, y_pred)
print("Precision sur l'ensemble de test:", precision)

###GRID SEARCH POUR ROC AUC###
grid_search_roc_auc = GridSearchCV(estimator=model, param_grid=param_grid, cv=k_fold, scoring='roc_auc')
grid_search_roc_auc.fit(X_train, y_train)
# Afficher les meilleurs paramètres et le score roc_auc
print("Best Parameters (ROC AUC):", grid_search_roc_auc.best_params_)
model_roc_auc = grid_search_roc_auc.best_estimator_

y_pred_proba = model_roc_auc.predict_proba(X_test)[:, 1]  # Probabilité de la classe positive
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC AUC sur l'ensemble de test:", roc_auc)
#Le coefficient de détermination R² est une métrique utilisée pour évaluer les modèles de régression, mais il n'est pas applicable aux modèles de classification comme XGBoost actue


Best Parameters (f1): {'colsample_bytree': 0.8, 'max_depth': 25, 'n_estimators': 200, 'subsample': 0.9}
Score F1 sur l'ensemble de test: 0.8713357046485956
Best Parameters (Accuracy): {'colsample_bytree': 0.8, 'max_depth': 25, 'n_estimators': 200, 'subsample': 0.9}
Accuracy sur l'ensemble de test: 0.9108599592114208
Best Parameters (Recall): {'colsample_bytree': 0.7, 'max_depth': 25, 'n_estimators': 200, 'subsample': 0.8}
Recall sur l'ensemble de test: 0.8592023065833734
Best Parameters (Precision): {'colsample_bytree': 0.7, 'max_depth': 25, 'n_estimators': 100, 'subsample': 0.9}
Precision sur l'ensemble de test: 0.8884442221110556
Best Parameters (ROC AUC): {'colsample_bytree': 0.8, 'max_depth': 25, 'n_estimators': 200, 'subsample': 0.9}
ROC AUC sur l'ensemble de test: 0.9542412297987262
